In [1]:
# Import dependencies
import pandas as pd
import openpyxl
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load data from excel
api_df = pd.read_excel("../data/raw/RiskClassification_Data_Endpoints_V1.xlsx", 
                       "Core_Endpoint",
                       usecols="A:R")
api_df.head()

,api_endpoint_id,api_id,api_vendor_id,api_vendor,api,category,usage_base,sample_response,tagset,authentication,security_test_category,security_test_result (FALSE=Passed; TRUE=Failed),server_location,hosting_isp,server_name,response_metadata,hosting city,risk_label_Baljeet
0,2410,1045,361,OneLook,Datamuse Dictionary,Research & Education,free,NaN,"tags,score,word,word,word",none,Broken Authentication,0.0,United States,"Amazon.com, Inc.",Apache/2.4.33 (Amazon) mod_wsgi/3.5 Python/2.7...,"{""Date"": ""Fri, 08 Nov 2019 14:43:17 GMT"", ""Var...",Ashburn,Low
1,2410,1045,361,OneLook,Datamuse Dictionary,Research & Education,free,NaN,"tags,score,word,word,word",none,Broken Authentication,0.0,United States,"Amazon.com, Inc.",Unavailable/Obscured,"{""Date"": ""Fri, 08 Nov 2019 14:43:17 GMT"", ""Var...",Ashburn,No
2,2410,1045,361,OneLook,Datamuse Dictionary,Research & Education,free,NaN,"tags,score,word,word,word",none,Cross-Site Scripting,0.0,United States,"Amazon.com, Inc.",Apache/2.4.33 (Amazon) mod_wsgi/3.5 Python/2.7...,"{""Date"": ""Fri, 08 Nov 2019 14:43:18 GMT"", ""Var...",Ashburn,Low
3,2410,1045,361,OneLook,Datamuse Dictionary,Research & Education,free,NaN,"tags,score,word,word,word",none,Cross-Site Scripting,0.0,United States,"Amazon.com, Inc.",Apache/2.4.33 (Amazon) mod_wsgi/3.5 Python/2.7...,NaN,Ashburn,Low
4,2410,1045,361,OneLook,Datamuse Dictionary,Research & Education,free,NaN,"tags,score,word,word,word",none,Cross-Site Scripting,0.0,United States,"Amazon.com, Inc.",Unavailable/Obscured,"{""Date"": ""Fri, 08 Nov 2019 14:43:18 GMT"", ""Var...",Ashburn,No


In [3]:
# Selecting relevant columns
security_test_df = api_df[["api_endpoint_id", "security_test_category", "security_test_result (FALSE=Passed; TRUE=Failed)"]]
security_test_df = security_test_df.rename(columns = {"api_endpoint_id":"endpoint_id",
                                   "security_test_category":"test_category",
                                    "security_test_result (FALSE=Passed; TRUE=Failed)":"test_result"})
security_test_df.head()

,endpoint_id,test_category,test_result
0,2410,Broken Authentication,0.0
1,2410,Broken Authentication,0.0
2,2410,Cross-Site Scripting,0.0
3,2410,Cross-Site Scripting,0.0
4,2410,Cross-Site Scripting,0.0


In [4]:
# List of unique test categories
list(security_test_df["test_category"].unique())

['Broken Authentication',
 'Cross-Site Scripting',
 'Insecure Deserialization',
 'SQL Injection',
 'XML External Entities',
 'Buffer Overflow',
 nan]

In [5]:
# Check presence of nan values
security_test_df[security_test_df["test_category"].isnull()]

,endpoint_id,test_category,test_result
271,2677,NaN,NaN
295,2687,NaN,NaN
296,2687,NaN,NaN
297,2687,NaN,NaN
298,2687,NaN,NaN
...,...,...,...
416,2687,NaN,NaN
417,2687,NaN,NaN
418,2687,NaN,NaN
419,2687,NaN,NaN


In [6]:
# Count number of records for each endpoint, including NaN values
security_test_df.groupby('endpoint_id').size()

endpoint_id
2410     16
2524      1
2532      1
2544     20
2545     60
2546      9
2548      2
2583      4
2585     34
2600     16
2601     13
2612     48
2628     30
2677     19
2681     22
2687    175
2699     10
2702    158
2730    229
2753      2
2754    284
2761      1
2795      5
2796      2
2812    574
2823    265
dtype: int64

In [7]:
# Cound number of records for each endpoint, excluding NaN values
feat_security_df = security_test_df.groupby('endpoint_id').count()
feat_security_df

,test_category,test_result
endpoint_id,,
2410,16,16
2524,1,1
2532,1,1
2544,20,20
2545,60,60
2546,9,9
2548,2,2
2583,4,4
2585,34,34


In [8]:
# Adding rows with no record of security tests
#security_count_df["api_id"] = security_count_df.index
#feat_security_test = pd.merge(feat_security_test, security_count_df, on="api_id", how="outer")

In [9]:
# Adding feature "security_test_conducted": 1 for tests-conducted, 0 for tests-not-conducted
feat_security_df["security_test_conducted"] = 0
feat_security_df.loc[feat_security_df["test_category"] > 0 , "security_test_conducted"] = 1
feat_security_df.drop(columns=["test_category", "test_result"], inplace=True)
feat_security_df.head()

,security_test_conducted
endpoint_id,
2410,1
2524,1
2532,1
2544,1
2545,1


In [10]:
# Adding features for each Security category "test_result":
# 1 for all tests FAIL, 0 for all tests PASS
stacked_df = security_test_df.groupby(["endpoint_id", "test_category"]).mean()
stacked_df

test_result
endpoint_id test_category                        
2410        Broken Authentication             0.0
            Cross-Site Scripting              0.0
            Insecure Deserialization          0.0
            SQL Injection                     0.0
            XML External Entities             0.0
...                                           ...
2796        Broken Authentication             1.0
2812        Insecure Deserialization          0.0
            SQL Injection                     1.0
2823        Buffer Overflow                   0.0
            Cross-Site Scripting              0.0

[64 rows x 1 columns]

In [11]:
stacked_df.unstack()

test_result                                       \
test_category Broken Authentication Buffer Overflow Cross-Site Scripting   
endpoint_id                                                                
2410                       0.000000             NaN                  0.0   
2524                       0.000000             NaN                  NaN   
2532                       1.000000             NaN                  NaN   
2544                            NaN             0.0                  NaN   
2545                       0.000000             NaN                  NaN   
2546                            NaN             NaN                  NaN   
2548                            NaN             NaN                  NaN   
2583                            NaN             NaN                  NaN   
2585                            NaN             NaN                  0.0   
2600                       0.000000             NaN                  0.0   
2601                       1.000000             NaN                  0.0   
2612                       1.000000             NaN                  0.0   
2628                            NaN             NaN                  0.0   
2677                            NaN             0.0                  0.0   
2681                       1.000000             NaN                  NaN   
2687                       1.000000             NaN                  NaN   
2699                            NaN             NaN                  NaN   
2702                       0.000000             NaN                  0.0   
2730                       0.000000             NaN                  0.0   
2753                            NaN             NaN                  NaN   
2754                       0.333333             NaN                  0.0   
2761                       0.000000             NaN                  NaN   
2795                       0.600000             NaN                  NaN   
2796                       1.000000             NaN                  NaN   
2812                            NaN             NaN                  NaN   
2823                            NaN             0.0                  0.0   

                                                                            
test_category Insecure Deserialization SQL Injection XML External Entities  
endpoint_id                                                                 
2410                               0.0      0.000000              0.000000  
2524                               NaN           NaN                   NaN  
2532                               NaN           NaN                   NaN  
2544                               NaN      0.285714                   NaN  
2545                               NaN      0.000000                   NaN  
2546                               NaN      0.000000                   NaN  
2548                               NaN      0.000000                   NaN  
2583                               0.0           NaN                   NaN  
2585                               NaN      0.533333                   NaN  
2600                               0.0      0.000000              0.000000  
2601                               NaN      0.000000              1.000000  
2612                               0.0      0.000000              1.000000  
2628                               NaN           NaN              0.000000  
2677                               NaN           NaN              0.500000  
2681                               NaN      0.000000                   NaN  
2687                               NaN      0.000000                   NaN  
2699                               NaN      0.000000                   NaN  
2702                               0.0      0.978261              0.727273  
2730                               0.0      0.000000              0.000000  
2753                               NaN      0.000000              1.000000  
2754                               0.0      0.00

In [12]:
# Combining Security_test_conducted and test_result features
feat_security_df = pd.merge(feat_security_df,
                            stacked_df.unstack(),
                            on="endpoint_id",
                            how="outer")
feat_security_df

,security_test_conducted,"(test_result, Broken Authentication)","(test_result, Buffer Overflow)","(test_result, Cross-Site Scripting)","(test_result, Insecure Deserialization)","(test_result, SQL Injection)","(test_result, XML External Entities)"
endpoint_id,,,,,,,
2410,1,0.000000,NaN,0.0,0.0,0.000000,0.000000
2524,1,0.000000,NaN,NaN,NaN,NaN,NaN
2532,1,1.000000,NaN,NaN,NaN,NaN,NaN
2544,1,NaN,0.0,NaN,NaN,0.285714,NaN
2545,1,0.000000,NaN,NaN,NaN,0.000000,NaN
2546,1,NaN,NaN,NaN,NaN,0.000000,NaN
2548,1,NaN,NaN,NaN,NaN,0.000000,NaN
2583,1,NaN,NaN,NaN,0.0,NaN,NaN
2585,1,NaN,NaN,0.0,NaN,0.533333,NaN


In [13]:
# Imputing missing values: if a test result is not available, fill '-1' as default
feat_security_df.fillna(-1, inplace=True)
feat_security_df.round(decimals=1)

,security_test_conducted,"(test_result, Broken Authentication)","(test_result, Buffer Overflow)","(test_result, Cross-Site Scripting)","(test_result, Insecure Deserialization)","(test_result, SQL Injection)","(test_result, XML External Entities)"
endpoint_id,,,,,,,
2410,1,0.0,-1.0,0.0,0.0,0.0,0.0
2524,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
2532,1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2544,1,-1.0,0.0,-1.0,-1.0,0.3,-1.0
2545,1,0.0,-1.0,-1.0,-1.0,0.0,-1.0
2546,1,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
2548,1,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
2583,1,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
2585,1,-1.0,-1.0,0.0,-1.0,0.5,-1.0


In [14]:
# Final Security Test features
feat_security_df.round(decimals=1)

,security_test_conducted,"(test_result, Broken Authentication)","(test_result, Buffer Overflow)","(test_result, Cross-Site Scripting)","(test_result, Insecure Deserialization)","(test_result, SQL Injection)","(test_result, XML External Entities)"
endpoint_id,,,,,,,
2410,1,0.0,-1.0,0.0,0.0,0.0,0.0
2524,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
2532,1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2544,1,-1.0,0.0,-1.0,-1.0,0.3,-1.0
2545,1,0.0,-1.0,-1.0,-1.0,0.0,-1.0
2546,1,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
2548,1,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
2583,1,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
2585,1,-1.0,-1.0,0.0,-1.0,0.5,-1.0
